In [1]:
# Using Ripser

import numpy as np
from ripser import ripser
from persim import plot_diagrams
import operator

## Persistence Landscapes: Algorithm 1

In [130]:
A = np.array([ [1.0, 5.0], [2.0, 8.0], [3.0, 4.0], [5.0, 9.0], [6.0, 7.0] ]) 
L = np.array([])
k = 0

# Sort A: read from right to left inside ()
ind =  np.lexsort((-A[:,1], A[:,0])) 
A = A[ind]

while len(A) != 0:
    print("k is",k)
    
    # pop first term
    bd, A = A[0], A[1:len(A)] 
    b, d = bd
    L = np.append(L, np.array([[-np.inf, 0], [b, 0], [(b+d)/2, (d-b)/2]]) ) # outer brackets for start of L_k
    
    print("L is",L)
    while L[-2] != np.inf or L[-1] != 0:

        # Check if d is greater than all remaining pairs
        if (d  > A[:,1]).all(): # check dont need vector
            
            # add to end of L_k
            L = np.append(L, np.array([ [d,0], [np.inf, 0] ]))

        else:
            # set (b', d')  to be the first term so that d' > d
            for i in range(len(A)):
                if A[i][1] > d: 
                    # pop (b', d')
                    
                    ind1 = [k for k in range(len(A) ) if k != i] 
                    
                    bd_prime, A = A[i], A[ind1]
                    
                    b_prime, d_prime = bd_prime
                    break
            print("(b',d') is", [b_prime, d_prime])

            # Case I    
            if b_prime > d:
                L = np.append(L, np.array([[d, 0] ]))

            # Case II
            if b_prime >= d:
                 L = np.append(L, np.array([ [b_prime, 0] ]))

            # Case III
            else:
                L = np.append(L, np.array([ [(b_prime + d)/2, (d-b_prime)/2] ]) ) 
                # Push (b', d) into A in order
                
                for i in range(len(A)):
                    if b_prime <= A[i][0]:
                        ind2 = i 
                        break
                        
                if b_prime == A[ind2][0]:
                    A_i = A[ A[:,0] == b_prime]
                    
                    for j in range(len(A_i)):
                        if d < A_i[j][1]:
                            ind2 = (ind2) + j
                            
                A = np.insert(A, ind2 ,np.array([b_prime, d]), axis = 0)


            L = np.append(L, np.array([ [(b_prime + d_prime)/2, (d_prime-b_prime)/2] ]) ) 
            b,d = b_prime, d_prime # Set (b',d')= (b, d)     
            print("A is", A)
    k += 1
    

k is 0
L is [-inf   0.   1.   0.   3.   2.]
(b',d') is [2.0, 8.0]
A is [[2. 5.]
 [3. 4.]
 [5. 9.]
 [6. 7.]]
(b',d') is [5.0, 9.0]
A is [[2. 5.]
 [3. 4.]
 [5. 8.]
 [6. 7.]]
k is 1
L is [-inf  0.   1.   0.   3.   2.   3.5  1.5  5.   3.   6.5  1.5  7.   2.
  9.   0.   inf  0.  -inf  0.   2.   0.   3.5  1.5]
(b',d') is [5.0, 8.0]
A is [[3. 4.]
 [6. 7.]]
k is 2
L is [-inf  0.   1.   0.   3.   2.   3.5  1.5  5.   3.   6.5  1.5  7.   2.
  9.   0.   inf  0.  -inf  0.   2.   0.   3.5  1.5  5.   0.   6.5  1.5
  8.   0.   inf  0.  -inf  0.   3.   0.   3.5  0.5]
(b',d') is [6.0, 7.0]
A is []


In [34]:
(np.array([1,2]) > np.array ([2,2])).all()

False

In [122]:
L = np.array([[0,0]])
L

array([[0, 0]])

In [123]:
L1= np.insert(L, 0, np.array([3,4]), axis = 0)
L1

array([[3, 4],
       [0, 0]])

In [124]:
L2= np.insert(L1, 0, np.array([[5,5]]), axis = 0)
L2

array([[5, 5],
       [3, 4],
       [0, 0]])

In [45]:
A[:,np.newaxis,:]

array([[[2., 8.]],

       [[3., 4.]],

       [[5., 9.]],

       [[6., 7.]]])

In [93]:
A = np.array([ [1.0, 5.0],[2.0,3], [2.0, 8.0], [3.0, 4.0], [5.0, 9.0], [6.0, 7.0] ]) 
A

array([[1., 5.],
       [2., 3.],
       [2., 8.],
       [3., 4.],
       [5., 9.],
       [6., 7.]])

In [97]:
np.insert(A, 5, np.array([3,3]), axis = 0)

array([[1., 5.],
       [2., 3.],
       [2., 8.],
       [3., 4.],
       [5., 9.],
       [3., 3.],
       [6., 7.]])

In [92]:
A[ A[:,0] == 2 ]

array([[2., 3.],
       [2., 8.]])

In [87]:
A[:,0] 

array([], dtype=float64)

In [75]:
A[0:1 & 2:len(A)]

array([], shape=(0, 2), dtype=float64)

In [68]:
(3 * np.ones(len(A)) > A[:,1]).all()

False

In [80]:
x = [k for k in range(len(A) ) if k !=1]
x

[0, 2, 3]

In [81]:
A[x]

array([[2., 8.],
       [5., 9.],
       [6., 7.]])

In [26]:
L

array([-inf,   0.,   1.,   0.,   3.,   2.])

In [53]:
L1=L.reshape((3,2))
L1

array([[-inf,   0.],
       [  1.,   0.],
       [  3.,   2.]])

In [59]:
np.append(L1, (np.array([4,4]) )

SyntaxError: unexpected EOF while parsing (<ipython-input-59-2344d178d312>, line 1)

In [58]:
np.array([4,4])[np.newaxis]

array([[4, 4]])

In [51]:
L[np.newaxis]

TypeError: 'NoneType' object is not callable

In [27]:
a = np.empty((len(L),))
a[::2] = 0
a[1::2] = 1

In [30]:
np.transpose(a)
a

array([0., 1., 0., 1., 0., 1.])

In [31]:
L * a

/opt/anaconda3/envs/Masters_Thesis/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in multiply
  """Entry point for launching an IPython kernel.


array([nan,  0.,  0.,  0.,  0.,  2.])

In [69]:
A = np.array([ [2.0, 8.0], [3.0, 4.0], [5.0, 9.0], [6.0, 7.0] ]) 
A = A.tolist() # turn numpy array into list

b,d = 1,5

In [73]:
## list comprehension example
[1 for i in range(10) if i >5]

[1, 1, 1, 1]

In [ ]:
bigger_than_list = [1 for i in range(len(A)) if d <= A[i][1]]
bigger_than_list == []

### Sorting using .sort

In [2]:
test1 =[  (2.0, 8.0), (4.0, 3.0), (5.0, 9.0), (6.0,9.0), (6.0, 7.0),(1.0, 5.0) ]
dtype =[('first_coord', float), ('second_coord', float)]
a = np.array(test1, dtype=dtype) 
a

array([(2., 8.), (4., 3.), (5., 9.), (6., 9.), (6., 7.), (1., 5.)],
      dtype=[('first_coord', '<f8'), ('second_coord', '<f8')])

Sort based on first coordinate, second coordinate sorted automatically increasing

In [4]:
sort1 = np.sort(a, order='first_coord')
sort1

array([(1., 5.), (2., 8.), (4., 3.), (5., 9.), (6., 7.), (6., 9.)],
      dtype=[('first_coord', '<f8'), ('second_coord', '<f8')])

Sorts both coordinates backwards

In [24]:
a[:: -1].sort(order = 'second_coord')
a

array([(6., 9.), (5., 9.), (2., 8.), (6., 7.), (1., 5.), (4., 3.)],
      dtype=[('first_coord', '<f8'), ('second_coord', '<f8')])

In [21]:
np.sort(a, order='first_coord')

array([(1., 5.), (2., 8.), (4., 3.), (5., 9.), (6., 7.), (6., 9.)],
      dtype=[('first_coord', '<f8'), ('second_coord', '<f8')])

In [12]:
temp = np.random.randint(1,10, 10)  
temp

array([4, 4, 1, 3, 4, 3, 7, 2, 3, 6])

In [14]:
temp[:: -1].sort()
temp

array([7, 6, 4, 4, 4, 3, 3, 3, 2, 1])

### Sorting using lexsort

In [48]:
a = np.array([[3, 2], [6, 2], [3, 6], [3, 4], [5, 3]])

ind1 =  np.lexsort((a[:,1], a[:,0])) 
ind2 =  np.lexsort((-a[:,1], a[:,0])) 
a

array([[3, 2],
       [6, 2],
       [3, 6],
       [3, 4],
       [5, 3]])

In [49]:
a[ind1]

array([[3, 2],
       [3, 4],
       [3, 6],
       [5, 3],
       [6, 2]])

In [50]:
a[ind2]

array([[3, 6],
       [3, 4],
       [3, 2],
       [5, 3],
       [6, 2]])

In [51]:
ind

array([2, 3, 0, 4, 1])